In [8]:
import zipfile
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

--2023-10-25 00:14:05--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.51, 3.163.189.96, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M   250MB/s    in 2.6s    

2023-10-25 00:14:07 (250 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [10]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

^C
unzip:  cannot find or open crawl-300d-2M.vec.zip, crawl-300d-2M.vec.zip.zip or crawl-300d-2M.vec.zip.ZIP.


In [11]:
!wget -O Sherlock.txt https://www.gutenberg.org/ebooks/48320.txt.utf-8

--2023-10-25 00:14:47--  https://www.gutenberg.org/ebooks/48320.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-25 00:14:47--  http://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-25 00:14:47--  https://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622314 (608K) [text/plain]
Saving to: ‘Sherlock.txt’

Sherlock.txt        100%[===================>] 607.73

In [12]:
with open("Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()

In [13]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|no|No)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = ['<SOS> ' + s.strip() + ' <EOS>' for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [14]:
book = split_into_sentences(book)

In [ ]:
book[1000:1010]

['<SOS> I am going through the City first, and we can have some lunch on the way. <EOS>',
 '<SOS> I observe that there is a good deal of German music on the programme, which is rather more to my taste than Italian or French. <EOS>',
 '<SOS> It is introspective, and I want to introspect. <EOS>',
 '<SOS> Come along! <EOS>',
 '<SOS> ”  We travelled by the Underground as far as Aldersgate; and a short walk took us to Saxe-Coburg Square, the scene of the singular story which we had listened to in the morning. <EOS>',
 '<SOS> It was a poky, little, shabby-genteel place, where four lines of dingy two-storied brick houses looked out into a small railed-in enclosure, where a lawn of weedy grass and a few clumps of faded laurel bushes made a hard fight against a smoke-laden and uncongenial atmosphere. <EOS>',
 '<SOS> Three gilt balls and a brown board with “JABEZ WILSON” in white letters, upon a corner house, announced the place where our red-headed client carried on his business. <EOS>',
 '<SOS

In [17]:
maxTokens = 5000
token = Tokenizer(num_words=maxTokens,
                  filters='!"“”#$%&()*+,.-/:;=?@[\\]^_`{|}~\t\n\ufeff\u2002', lower=True,
                  split=' ', char_level=False, oov_token="")
token.fit_on_texts(book)

In [18]:
sequences = token.texts_to_sequences(book)

In [ ]:
textSequences = token.sequences_to_texts(sequences)

In [19]:
maxLen = len(max(sequences, key=len))

In [20]:
def create_dataset(sequence):
    inputs = []
    outputs = []
    for i, sent in enumerate(sequence):
        for j in range(1, len(sent)-1):
          inputs.append(sent[:j])
          outputs.append([sent[j]])

    return inputs, outputs

In [21]:
X, y = create_dataset(sequences)

In [22]:
token.sequences_to_texts([X[15]])

['<sos> the project gutenberg ebook of adventures of sherlock holmes this ebook is for the use']

In [23]:
maxLen = len(max(X, key=len))
x_train = pad_sequences(X, maxlen=maxLen)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])
num_words=len(dictionary)+1
y_train = np.array(tf.keras.utils.to_categorical(y, num_classes=num_words))

In [24]:
import codecs
import IPython.display as ipd
from IPython.display import Audio, update_display
from ipywidgets import IntProgress

#EMB = "crawl-300d-2M.vec"
EMB = "wiki-news-300d-1M.vec"
N = 1000000

#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open(f'./{EMB}', encoding='utf-8')
bar = IntProgress(min=0, max=N)
ipd.display(bar)

i = 0
for n, line in enumerate(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if n//(N//100) > i:
          bar.value = n
          i += 1
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


IntProgress(value=0, max=1000000)

found 999995 word vectors


In [25]:
embed_dim=300
embedding_matrix=np.zeros([num_words, embed_dim])
for word, idx in dictionary.items():
  if word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3)

In [27]:
import pickle

FILE = './var.pick'
with open(FILE, 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump([embedding_matrix, x_train, x_test, y_train, y_test], file)

# RAM Checkpoint

In [1]:
import pickle

FILE = './var.pick'
with open(FILE, 'rb') as file:
    # Deserialize and retrieve the variable from the file
    embedding_matrix, x_train, x_test, y_train, y_test = pickle.load(file)

In [2]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def softMaxOverTime(x):
    return softmax(x, axis=0)

nb_words = len(embedding_matrix) #Vocabulary size
embed_dim = len(embedding_matrix[0]) #Vectorization dim
maxLen = len(x_train[0])
valueDim = 32 #Cantidad de filtros del contextualizador

In [3]:
input_layer = Input(shape=(maxLen, )) #Entrada de texto de largo 500 maximo.

#Creamos un embedding de len(vocab) x tamaño de la representacion de cada palabra, usando el embedding descargado como pesos de la representacion de las palabras, con un largo maximo como entrada.
embedding_layer = Embedding(nb_words, embed_dim, weights = [embedding_matrix], input_length = maxLen, trainable = True)(input_layer)

#Contextualizador, aca esta lo que proyecta el vector de entrada con el query que son los pesos de los filtros.
conv_out = Conv1D(filters = valueDim, kernel_size = 8, activation = "tanh", padding = "same")(embedding_layer)
conv_out = BatchNormalization()(conv_out)
conv_out = Conv1D(filters = valueDim, kernel_size = 4, activation = "tanh", padding = "same")(conv_out)
conv_out = BatchNormalization()(conv_out)
conv_out = Dropout(0.3)(conv_out)
ulog_attention = Dense(1, activation="linear")(conv_out)

#La parte de attention
attention = Activation(softMaxOverTime)(ulog_attention)
repeated_attention = TimeDistributed(RepeatVector(valueDim))(attention)
repeated_attention = Reshape([maxLen, valueDim])(repeated_attention)

#Aca esta la parte de value
weighted_embeddings = Multiply()([repeated_attention, conv_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)

#Salida
dense2 = Dense(nb_words, activation='softmax')(embedding_sum)
model = Model(input_layer, dense2)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 172)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 172, 300)             2653200   ['input_1[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 172, 32)              76832     ['embedding_1[0][0]']         
                                                                                                  
 batch_normalization (Batch  (None, 172, 32)              128       ['conv1d_1[0][0]']            
 Normalization)                                                                               

In [4]:
from tensorflow.keras import losses, backend
import tensorflow as tf

def perplexity(y_true, y_pred):
    """
    The perplexity metric. Why isn't this part of Keras yet?!
    https://stackoverflow.com/questions/41881308/how-to-calculate-perplexity-of-rnn-in-tensorflow
    https://github.com/keras-team/keras/issues/8267
    """
    cross_entropy = losses.categorical_crossentropy(y_true, y_pred)
    perplexity = tf.pow(2.0, cross_entropy)
    return perplexity

In [5]:
opt = optimizers.Adam(learning_rate=0.01)
callbackROP = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)
callbackES = EarlyStopping(monitor='val_loss', patience=10)

In [6]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=[perplexity])

In [7]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=1024, epochs=50, callbacks=[callbackROP, callbackES])

Epoch 1/50
76/76 [==============================] - ETA: 0s - loss: 8.9035 - perplexity: 325269.9375

KeyboardInterrupt: ignored

In [8]:
aver = token.sequences_to_texts([[2599]])
aver

NameError: ignored

In [8]:
question = token.texts_to_sequences(["<sos>"])

NameError: ignored

In [26]:
question = pad_sequences(question, maxlen=maxLen)

# Greedy search

In [27]:
response = []
reply = question
out = 0
finished = False

while out != token.texts_to_sequences(["<eos>"])[0][0] and not finished:
    out = pad_sequences(reply, maxlen=maxLen)
    out = model.predict(out, verbose=0)
    out = int(np.argmax(out))
    reply = [[out]]
    if out in response:
      finished = True
    else:
      response.append(out)

In [28]:
def from_seq_to_text(seq):
    return token.sequences_to_texts([seq])

In [29]:
rta = from_seq_to_text(response)
rta

['the']